# Replace team abbreviations with proper abbreviations
Related to \#1, bug in code recorded incorrect team abbreviations

In [22]:
import sys
import os
import matplotlib.pyplot as plt
% matplotlib notebook
import pickle
# Import class
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import NBAStats
import datetime as dt

In [9]:
stats_file = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
print(stats_file)
nba_stats = pickle.load(open(stats_file, 'rb'))

C:\Users\johnn\Desktop\github\nba_stats\Data\nba_stats.pickle


In [10]:
dict_player_stats = nba_stats.get_player_stats()
games = nba_stats.get_games().sort_values(by='GAME_DATE')
df_player = dict_player_stats[203967]

In [11]:
games.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID,OPP_PTS
0,2011-12,1610612759,SAS,0041100141,2012-04-29T00:00:00,SAS vs. UTA,W,106,HOME,UTA,1610612762,91
2,2011-12,1610612737,ATL,0041100131,2012-04-29T00:00:00,ATL vs. BOS,W,83,HOME,BOS,1610612738,74
3,2011-12,1610612746,LAC,0041100171,2012-04-29T00:00:00,LAC @ MEM,W,99,AWAY,MEM,1610612763,98
4,2011-12,1610612763,MEM,0041100171,2012-04-29T00:00:00,MEM vs. LAC,L,98,HOME,LAC,1610612746,99
5,2011-12,1610612738,BOS,0041100131,2012-04-29T00:00:00,BOS @ ATL,L,74,AWAY,ATL,1610612737,83


In [12]:
df_player.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-16,0021800001,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,22:54,3,...,1,0,0,3,5,6,-4,AWAY,BOS,1610612738
2018-10-18,0021800014,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,28:20,4,...,3,1,0,3,2,13,9,HOME,CHI,1610612741
2018-10-20,0021800029,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:34,4,...,5,1,1,2,3,13,-11,HOME,PHI,1610612753
2018-10-23,0021800049,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:42,5,...,5,1,1,1,4,14,1,AWAY,DET,1610612765
2018-10-24,0021800059,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,33:30,6,...,0,0,0,1,3,15,-16,AWAY,PHI,1610612749


### Create mapping
We need a mapping from TEAM_ID to TEAM_ABBREVIATION

In [13]:
df_team_id_abb = games[['GAME_DATE', 'TEAM_ID', 'TEAM_ABBREVIATION']]
df_team_id_abb_unique = df_team_id_abb.drop_duplicates(subset='TEAM_ABBREVIATION').sort_values(by=['TEAM_ID', 
                                                                                                  'GAME_DATE'])
# Convert GAME_DATE from str to dt.date
df_team_id_abb_unique.GAME_DATE = df_team_id_abb_unique.GAME_DATE.apply(
    lambda date: dt.datetime.strptime(date[:10], '%Y-%m-%d'))

In [14]:
df_team_id_abb_unique.head(6)

,GAME_DATE,TEAM_ID,TEAM_ABBREVIATION
2,2012-04-29,1610612737,ATL
5,2012-04-29,1610612738,BOS
14,2015-01-02,1610612739,CLE
21,2015-01-02,1610612740,NOP
0,2015-01-01,1610612741,CHI
19,2015-01-02,1610612742,DAL


### Create function to map
Create a function to run over all entries in player stats

In [15]:
def team_id_to_abb(team_id, mapping, date):
    df_team_abbs = mapping[(mapping.TEAM_ID == team_id) &
                      (mapping.GAME_DATE <= date)]
    most_rec_date = max(df_team_abbs.GAME_DATE)
    team_abb = df_team_abbs[df_team_abbs.GAME_DATE == most_rec_date]['TEAM_ABBREVIATION'].iloc[0]
    return team_abb

In [16]:
team_id_to_abb(1610612753, df_team_id_abb_unique, dt.date(2018, 10, 20))

'ORL'

### Create function to apply
Takes in a row of player df and returns the opponent's team abbreviation

In [17]:
def row_to_abb(row):
    date = row.name
    opp_team_id = row.OPP_TEAM_ID
    opp_team_abb = team_id_to_abb(opp_team_id, df_team_id_abb_unique, date)
    return opp_team_abb

In [18]:
df_player.OPP_TEAM_ABBREVIATION = df_player.apply(row_to_abb, axis=1)
df_player.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-16,0021800001,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,22:54,3,...,1,0,0,3,5,6,-4,AWAY,BOS,1610612738
2018-10-18,0021800014,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,28:20,4,...,3,1,0,3,2,13,9,HOME,CHI,1610612741
2018-10-20,0021800029,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:34,4,...,5,1,1,2,3,13,-11,HOME,ORL,1610612753
2018-10-23,0021800049,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:42,5,...,5,1,1,1,4,14,1,AWAY,DET,1610612765
2018-10-24,0021800059,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,33:30,6,...,0,0,0,1,3,15,-16,AWAY,MIL,1610612749


In [19]:
finished = 1
total = len(dict_player_stats.keys())

for player_key in dict_player_stats.keys():
    df_player = nba_stats.get_player_stats()[player_key]
    df_player.OPP_TEAM_ABBREVIATION = df_player.apply(row_to_abb, axis=1)
    
    if finished % 5 == 0:
        print('updated player {} of {}'.format(finished, total))
    finished += 1


updated player 5 of 899
updated player 10 of 899
updated player 15 of 899
updated player 20 of 899
updated player 25 of 899
updated player 30 of 899
updated player 35 of 899
updated player 40 of 899
updated player 45 of 899
updated player 50 of 899
updated player 55 of 899
updated player 60 of 899
updated player 65 of 899
updated player 70 of 899
updated player 75 of 899
updated player 80 of 899
updated player 85 of 899
updated player 90 of 899
updated player 95 of 899
updated player 100 of 899
updated player 105 of 899
updated player 110 of 899
updated player 115 of 899
updated player 120 of 899
updated player 125 of 899
updated player 130 of 899
updated player 135 of 899
updated player 140 of 899
updated player 145 of 899
updated player 150 of 899
updated player 155 of 899
updated player 160 of 899
updated player 165 of 899
updated player 170 of 899
updated player 175 of 899
updated player 180 of 899
updated player 185 of 899
updated player 190 of 899
updated player 195 of 899
update

In [20]:
nba_stats.get_player_stats()[203967]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,HOME_AWAY,OPP_TEAM_ABBREVIATION,OPP_TEAM_ID
Date,,,,,,,,,,,,,,,,,,,,,
2018-10-16,0021800001,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,22:54,3,...,1,0,0,3,5,6,-4,AWAY,BOS,1610612738
2018-10-18,0021800014,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,28:20,4,...,3,1,0,3,2,13,9,HOME,CHI,1610612741
2018-10-20,0021800029,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:34,4,...,5,1,1,2,3,13,-11,HOME,ORL,1610612753
2018-10-23,0021800049,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,37:42,5,...,5,1,1,1,4,14,1,AWAY,DET,1610612765
2018-10-24,0021800059,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,33:30,6,...,0,0,0,1,3,15,-16,AWAY,MIL,1610612749
2018-10-27,0021800078,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,30:33,4,...,1,0,0,1,4,11,0,HOME,CHA,1610612766
2018-10-29,0021800088,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,26:08,1,...,1,1,0,1,1,4,16,HOME,ATL,1610612737
2018-10-30,0021800100,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,31:37,3,...,2,0,0,3,2,8,-6,AWAY,TOR,1610612761
2018-11-01,0021800113,1610612755,PHI,Philadelphia,203967,Dario Saric,F,,24:17,1,...,2,0,0,1,5,5,-4,HOME,LAC,1610612746


# Resave new version

In [23]:
import pickle
import os

file_directory = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
pickle.dump(nba_stats, open(file_directory, 'wb'))